<a href="https://colab.research.google.com/github/IsamAljawarneh/Geo-Movers-Distance-GMD/blob/master/Geo_Movers_Distance_GMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Geo Mover's Distance GMD***
- Author: Dr. Isam Al Jawarneh

This notebook shows a running example of Geo Mover's Distance (GMD)

## preparation

In [ ]:
!pip install folium
!pip install uszipcode
%pip install pygeohash
!pip install polygeohasher
!pip install geopandas


In [2]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from IPython.display import Image
import folium
from folium import IFrame
from folium.plugins import MarkerCluster
from folium import plugins
from datetime import datetime
import datetime as dt
import json
from scipy import stats

import os
import pygeohash as gh

#from polygeohasher import polygeohasher
import geopandas as gpd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# END preparation

# config

In [3]:
geohash_precision = 6

# 2) bring raw data

Mount the drive so that you can read the data from google drive

In [4]:
trips = pd.read_csv('https://raw.githubusercontent.com/IsamAljawarneh/Geo-Movers-Distance-GMD/refs/heads/master/data/guang.csv')

In [ ]:
#trips.info()

In [5]:
# renaming as longitude latitude columns are reversed in the original data
trips =trips.rename(columns={'lat': 'longitude', 'lon': 'latitude'})


2.5 Convert datetime object column to datetime series

In [6]:
trips['time'] = pd.to_datetime(\
                trips['time'],dayfirst = True)
trips['time'] = pd.to_datetime(\
                trips['time'],dayfirst = True)


<ipython-input-6-f9311bbb90a4>:1: UserWarning: Parsing dates in %Y-%m-%dT%H:%M:%S.%f%z format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  trips['time'] = pd.to_datetime(\


add columns month, time, day

In [7]:
trips['Month'] = trips['time'].dt.month
trips['Time'] = trips['time'].dt.time
trips['Day'] = trips['time'].dt.day

In [8]:
trips.head(2)

,id,longitude,latitude,time,speed,Month,Time,Day
0,0,114.031799,22.524799,2014-10-22 02:54:30+00:00,42,10,02:54:30,22
1,0,114.038696,22.531500,2014-10-22 02:54:37+00:00,52,10,02:54:37,22


In [9]:
trips[(trips['Day']>2)].head(2)

,id,longitude,latitude,time,speed,Month,Time,Day
0,0,114.031799,22.524799,2014-10-22 02:54:30+00:00,42,10,02:54:30,22
1,0,114.038696,22.531500,2014-10-22 02:54:37+00:00,52,10,02:54:37,22


In [ ]:
#trips.info()

2.6 Remove erroneous coordinates (0,0) from the dataset


In [10]:
trips = \
trips[(trips['latitude'] != 0 ) & \
(trips['longitude']!=0 )]

return the size

In [11]:
trips.shape[0]

1155653

## Geohash

generate geohash for each tuple (long, lat)

In [12]:
trips['geohash']=trips.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=geohash_precision), axis=1)

In [13]:
%%timeit
trips.head(2)

37.3 µs ± 3.88 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [14]:
trips.shape[0]

1155653

In [15]:
trips.head(2)

,id,longitude,latitude,time,speed,Month,Time,Day,geohash
0,0,114.031799,22.524799,2014-10-22 02:54:30+00:00,42,10,02:54:30,22,ws104u
1,0,114.038696,22.531500,2014-10-22 02:54:37+00:00,52,10,02:54:37,22,ws105j


In [17]:
geohash_level = 6

In [19]:
pickup_geos = trips.groupby(['geohash']).size().reset_index()

In [20]:
pickup_geos.head(5)

,geohash,0
0,webzz9,6
1,webzzd,6
2,webzzf,17
3,webzzg,43
4,webzzm,4


we need to convert both trips and polygons to GeoPandas

In [21]:
trips.shape

(1155653, 9)

In [ ]:
#trips.dtypes

In [22]:
import geopandas as gpd


In [23]:
# 1 - convert to Geopandas Geodataframe
gdf_trips = gpd.GeoDataFrame(trips,   geometry=gpd.points_from_xy(trips.longitude, trips.latitude))

In [24]:
gdf_trips.shape

(1155653, 10)

it additionally trips now has the geometry column

In [25]:
gdf_trips.head(2)

,id,longitude,latitude,time,speed,Month,Time,Day,geohash,geometry
0,0,114.031799,22.524799,2014-10-22 02:54:30+00:00,42,10,02:54:30,22,ws104u,POINT (114.03180 22.52480)
1,0,114.038696,22.531500,2014-10-22 02:54:37+00:00,52,10,02:54:37,22,ws105j,POINT (114.03870 22.53150)





### now read polygons




In [28]:
# 2 - Neighbourhoods
geojson_file = "https://raw.githubusercontent.com/IsamAljawarneh/Geo-Movers-Distance-GMD/refs/heads/master/data/shenzhen_converted.geojson"
neighborhoods = gpd.read_file(geojson_file)

In [29]:
neighborhoods.head(2)

,OBJECTID_1,OBJECTID,NAME,CODE,Shape_Leng,Shape_Le_1,Shape_Area,geometry
0,1,1,龙岗区,440307,275239.988224,2.580133,0.083089,"POLYGON ((114.34860 22.59273, 114.34562 22.599..."
1,2,2,福田区,440304,41451.084303,0.390022,0.007023,"POLYGON ((114.06118 22.58837, 114.06147 22.587..."


In [30]:
gdf_trips.head(2)

,id,longitude,latitude,time,speed,Month,Time,Day,geohash,geometry
0,0,114.031799,22.524799,2014-10-22 02:54:30+00:00,42,10,02:54:30,22,ws104u,POINT (114.03180 22.52480)
1,0,114.038696,22.531500,2014-10-22 02:54:37+00:00,52,10,02:54:37,22,ws105j,POINT (114.03870 22.53150)


Keeping geometry column from both dataframes when applying sjoin() using GeoPandas


In [31]:
neighborhoods['geometryn'] = neighborhoods.geometry.to_crs("epsg:3857")

In [32]:
gdf_trips = gdf_trips.set_crs('epsg:4326')

In [ ]:
#gdf_trips.crs

In [ ]:
#neighborhoods.crs

In [ ]:
#neighborhoods.geometryn.crs

In [33]:

sjoined_trips = gpd.sjoin(gdf_trips, neighborhoods, op="within")
sjoined_trips.head(2)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


,id,longitude,latitude,time,speed,Month,Time,Day,geohash,geometry,index_right,OBJECTID_1,OBJECTID,NAME,CODE,Shape_Leng,Shape_Le_1,Shape_Area,geometryn
0,0,114.031799,22.524799,2014-10-22 02:54:30+00:00,42,10,02:54:30,22,ws104u,POINT (114.03180 22.52480),1,2,2,福田区,440304,41451.084303,0.390022,0.007023,"POLYGON ((12697232.672 2582314.409, 12697264.4..."
1,0,114.038696,22.531500,2014-10-22 02:54:37+00:00,52,10,02:54:37,22,ws105j,POINT (114.03870 22.53150),1,2,2,福田区,440304,41451.084303,0.390022,0.007023,"POLYGON ((12697232.672 2582314.409, 12697264.4..."


In [34]:
sjoined_trips.shape

(1154855, 19)

In [35]:
type(sjoined_trips)

geopandas.geodataframe.GeoDataFrame

## stratified sampling sample different percentages

In [36]:
sample_info_scenario1 = pd.DataFrame([['福田区',0.2],['南山区',0.2],['罗湖区',0.2],['宝安区',0.2],['龙岗区',0.2],['盐田区',0.2]], columns = ['NAME','SampleSize'])

In [ ]:
#scenario_2_disp_comb002_002_002_99_99__99

In [37]:
mapper_scenario1 = sample_info_scenario1.set_index('NAME')['SampleSize'].to_dict()

sampled_data_scenario1 = sjoined_trips.groupby('NAME').apply(lambda x: x.sample(frac=mapper_scenario1.get(x.name))).reset_index(drop = True)

<ipython-input-37-c0762051350d>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_data_scenario1 = sjoined_trips.groupby('NAME').apply(lambda x: x.sample(frac=mapper_scenario1.get(x.name))).reset_index(drop = True)


In [38]:
sample_info_scenario2 = pd.DataFrame([['福田区',0.9],['南山区',0.3],['罗湖区',0.9],['宝安区',0.9],['龙岗区',0.9],['盐田区',0.9]], columns = ['NAME','SampleSize'])

In [39]:
mapper_scenario2 = sample_info_scenario2.set_index('NAME')['SampleSize'].to_dict()

sampled_data_scenario2 = sjoined_trips.groupby('NAME').apply(lambda x: x.sample(frac=mapper_scenario2.get(x.name))).reset_index(drop = True)

<ipython-input-39-df045656e797>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_data_scenario2 = sjoined_trips.groupby('NAME').apply(lambda x: x.sample(frac=mapper_scenario2.get(x.name))).reset_index(drop = True)


In [40]:
sampled_data_scenario1.shape[0]

230971

In [41]:
sampled_data_scenario2.shape[0]

902807

# choropleth map generation

In [42]:
# sampled data scenario #1
shenzhen_taxi_pickup_sample= sampled_data_scenario1['NAME'].value_counts()
shenzhen_taxi_pickup_sample = shenzhen_taxi_pickup_sample.reset_index()
shenzhen_taxi_pickup_sample.columns = ['NAME','count']
shenzhen_taxi_pickup_sample['NAME'] = shenzhen_taxi_pickup_sample['NAME'].astype(str)

In [43]:
# sampled data scenario #2
shenzhen_taxi_pickup_sample2= sampled_data_scenario2['NAME'].value_counts()
shenzhen_taxi_pickup_sample2 = shenzhen_taxi_pickup_sample2.reset_index()
shenzhen_taxi_pickup_sample2.columns = ['NAME','count']
shenzhen_taxi_pickup_sample2['NAME'] = shenzhen_taxi_pickup_sample2['NAME'].astype(str)

In [44]:
# original data
shenzhen_taxi_pickup_original= sjoined_trips['NAME'].value_counts()
shenzhen_taxi_pickup_original = shenzhen_taxi_pickup_original.reset_index()
shenzhen_taxi_pickup_original.columns = ['NAME','count']
shenzhen_taxi_pickup_original['NAME'] = shenzhen_taxi_pickup_original['NAME'].astype(str)

In [45]:
shenzhen_taxi_pickup_sample2.head(6)

,NAME,count
0,福田区,474657
1,罗湖区,192505
2,宝安区,113747
3,南山区,68281
4,龙岗区,47214
5,盐田区,6403


In [46]:
shenzhen_taxi_pickup_original.head(6)

,NAME,count
0,福田区,527397
1,南山区,227604
2,罗湖区,213894
3,宝安区,126386
4,龙岗区,52460
5,盐田区,7114


In [61]:
shenzhen_taxi_pickup_original["NAME"].astype(str)
geo_path = r'https://raw.githubusercontent.com/IsamAljawarneh/Geo-Movers-Distance-GMD/refs/heads/master/data/shenzhen_converted.geojson'
heatmap_scale = list()
threshold = [10,20,50,70,85,100]
for i in threshold :
    heatmap_scale.append(int(shenzhen_taxi_pickup_original['count'].max() * (i/100.0)))

map_shenzhen_taxi_pickup_sample = folium.Map(location=[22.542883, 114.062996], zoom_start=10)
folium.Choropleth(geo_data=geo_path, data=shenzhen_taxi_pickup_original, \
                data_out = 'nyc_zip_test.json',
             columns=['NAME', 'count'],
             #threshold_scale= heatmap_scale,
             key_on='feature.properties.NAME',
             fill_color='PuBuGn', fill_opacity=0.9, line_opacity=0.9,
             legend_name='Number of Pickups').add_to(map_shenzhen_taxi_pickup_sample)

In [62]:
map_shenzhen_taxi_pickup_sample

In [64]:
## second scenario
shenzhen_taxi_pickup_sample2["NAME"].astype(str)
geo_path = r'https://raw.githubusercontent.com/IsamAljawarneh/Geo-Movers-Distance-GMD/refs/heads/master/data/shenzhen_converted.geojson'
heatmap_scale = list()
threshold = [10,20,50,70,85,100]
for i in threshold :
    heatmap_scale.append(int(shenzhen_taxi_pickup_sample2['count'].max() * (i/100.0)))

map_shenzhen_taxi_pickup_sample = folium.Map(location=[22.542883, 114.062996], zoom_start=10)
folium.Choropleth(geo_data=geo_path, data=shenzhen_taxi_pickup_sample2, \
                data_out = 'nyc_zip_test.json',
             columns=['NAME', 'count'],
             #threshold_scale= heatmap_scale,
             key_on='feature.properties.NAME',
             fill_color='YlGnBu', fill_opacity=0.9, line_opacity=0.9,
             legend_name='Number of Pickups').add_to(map_shenzhen_taxi_pickup_sample)

In [65]:
map_shenzhen_taxi_pickup_sample

# END choropleth map generation

### Geo Mover's Distance (GMD)

Using the Geo Mover's distance to quantify the similarity between two maps


In [66]:
import cv2

In [67]:
def signature_opt1(gdf, crs):

    centroids = gdf.geometry.to_crs(crs)#.centroid
    sig = np.empty((len(gdf), 3), dtype=np.float32)
    # float32 needed as input for cv2.emd!

    # we need to normalize the data in case the total
    # n of the two compared distributions are not equal
    #sig[:,0] = gdf.geohash /gdf.geohash.sum()
    #mean instead of count
    sig[:,0] = gdf.geohash /gdf.geohash.sum()
    #    sig[:,0] = gdf.speed # /gdf.speed.mean()
    print(np.sum(sig[:,0]))
    sig[:,1] = centroids.x
    sig[:,2] = centroids.y
    return sig

In [ ]:
#gdf_trips.dtypes

In [81]:
type(gdf_trips)

geopandas.geodataframe.GeoDataFrame

In [82]:
gdf_trips.head(1)

,id,longitude,latitude,time,speed,Month,Time,Day,geohash,geometry
0,0,114.031799,22.524799,2014-10-22 02:54:30+00:00,42,10,02:54:30,22,ws104u,POINT (114.03180 22.52480)


In [83]:
#sampled_data_original_grouped = gdf_trips[['geometry','geohash']].groupby(['geohash'], as_index=False).agg({'geohash': 'count', 'geometry': 'first'})

sampled_data_original_grouped = gdf_trips[['geometry','geohash']].groupby(['geohash'], as_index=False).agg({'geohash': 'count', 'geometry': 'first'})


In [ ]:
#gdf_trips.dtypes

In [ ]:
#type(sjoined_trips)

In [84]:
sjoined_trips.head(1)

,id,longitude,latitude,time,speed,Month,Time,Day,geohash,geometry,...,OBJECTID_1,OBJECTID,NAME,CODE,Shape_Leng,Shape_Le_1,Shape_Area,geometryn,centroid,otherNAME
0,0,114.031799,22.524799,2014-10-22 02:54:30+00:00,42,10,02:54:30,22,ws104u,POINT (114.03180 22.52480),...,2,2,福田区,440304,41451.084303,0.390022,0.007023,"POLYGON ((12697232.672 2582314.409, 12697264.4...",POINT (12695031.079 2577223.135),福田区


In [ ]:
#sjoined_trips.crs

In [86]:
sjoined_trips["centroid"] = sjoined_trips["geometryn"].centroid

In [87]:
sjoined_trips.head(1)

,id,longitude,latitude,time,speed,Month,Time,Day,geohash,geometry,...,OBJECTID_1,OBJECTID,NAME,CODE,Shape_Leng,Shape_Le_1,Shape_Area,geometryn,centroid,otherNAME
0,0,114.031799,22.524799,2014-10-22 02:54:30+00:00,42,10,02:54:30,22,ws104u,POINT (114.03180 22.52480),...,2,2,福田区,440304,41451.084303,0.390022,0.007023,"POLYGON ((12697232.672 2582314.409, 12697264.4...",POINT (12695031.079 2577223.135),福田区


In [88]:
sjoined_trips['otherNAME'] = sjoined_trips['NAME']

In [89]:
sjoined_trips.head(2)

,id,longitude,latitude,time,speed,Month,Time,Day,geohash,geometry,...,OBJECTID_1,OBJECTID,NAME,CODE,Shape_Leng,Shape_Le_1,Shape_Area,geometryn,centroid,otherNAME
0,0,114.031799,22.524799,2014-10-22 02:54:30+00:00,42,10,02:54:30,22,ws104u,POINT (114.03180 22.52480),...,2,2,福田区,440304,41451.084303,0.390022,0.007023,"POLYGON ((12697232.672 2582314.409, 12697264.4...",POINT (12695031.079 2577223.135),福田区
1,0,114.038696,22.531500,2014-10-22 02:54:37+00:00,52,10,02:54:37,22,ws105j,POINT (114.03870 22.53150),...,2,2,福田区,440304,41451.084303,0.390022,0.007023,"POLYGON ((12697232.672 2582314.409, 12697264.4...",POINT (12695031.079 2577223.135),福田区


In [95]:
## ALTERNATIVE : Group by NAME (district)
sampled_data_original_grouped = sjoined_trips[['otherNAME','centroid','NAME','geometry']].groupby(['NAME'], as_index=False).agg({'otherNAME':'first','NAME': 'count', 'centroid':'first' ,'geometry':'first'})


In [96]:
sampled_data_original_grouped.head(2)

,otherNAME,NAME,centroid,geometry
0,南山区,227604,POINT (12683172.678 2577481.685),POINT (113.97970 22.52290)
1,宝安区,126386,POINT (12680354.419 2595697.733),POINT (114.03900 22.60770)


In [ ]:
## ALTERNATIVE mean instead of count
#sampled_data_scenario1_grouped = gdf_trips[['geometry','geohash','speed']].groupby(['geohash'], as_index=False).agg({'speed': 'mean', 'geometry': 'first'})


In [97]:
sampled_data_original_grouped  = gpd.GeoDataFrame(sampled_data_original_grouped, crs="EPSG:3857", geometry=sampled_data_original_grouped.geometry)

In [98]:
sampled_data_original_grouped.dtypes

,0
otherNAME,object
NAME,int64
centroid,geometry
geometry,geometry


In [99]:
np.sum(sampled_data_original_grouped.NAME)

1154855

In [100]:
sampled_data_original_grouped.shape

(6, 4)

In [101]:
sampled_data_original_grouped.sort_values('NAME',ascending=False)

,otherNAME,NAME,centroid,geometry
3,福田区,527397,POINT (12695031.079 2577223.135),POINT (114.032 22.525)
0,南山区,227604,POINT (12683172.678 2577481.685),POINT (113.980 22.523)
4,罗湖区,213894,POINT (12706355.523 2581018.763),POINT (114.100 22.562)
1,宝安区,126386,POINT (12680354.419 2595697.733),POINT (114.039 22.608)
5,龙岗区,52460,POINT (12729987.892 2588957.163),POINT (114.111 22.598)
2,盐田区,7114,POINT (12720500.796 2583286.511),POINT (114.216 22.557)


In [102]:
type(sampled_data_original_grouped)

geopandas.geodataframe.GeoDataFrame

In [ ]:
#extract column count geohash to numpy array
#count_original = sampled_data_original_grouped['geohash'].to_numpy()
#count_original

In [103]:
#extract column count NAME to numpy array
count_original = sampled_data_original_grouped['NAME'].to_numpy()
count_original

array([227604, 126386,   7114, 527397, 213894,  52460])

In [104]:
# you need to normalize so that all add up to one
count_original = count_original/count_original.sum()
count_original

array([0.19708448, 0.10943885, 0.00616008, 0.45667811, 0.18521286,
       0.04542562])

In [105]:
shenzhen_taxi_pickup_original.dtypes

,0
NAME,object
count,int64


In [106]:
type(shenzhen_taxi_pickup_original)

pandas.core.frame.DataFrame

In [107]:
shenzhen_taxi_pickup_original.dtypes

,0
NAME,object
count,int64


In [108]:
##sampled_data_scenario_grouped = sampled_data_scenario2[['geometry','geohash']].groupby(['geohash'], as_index=False).agg({'geohash': 'count', 'geometry': 'first'})


In [109]:
sampled_data_scenario1.dtypes

,0
id,int64
longitude,float64
latitude,float64
time,"datetime64[ns, UTC]"
speed,int64
Month,int32
Time,object
Day,int32
geohash,object
geometry,geometry


In [110]:
sampled_data_scenario2['otherNAME'] = sampled_data_scenario2['NAME']

In [111]:
sampled_data_scenario2["centroid"] = sampled_data_scenario2["geometryn"].centroid

In [112]:
sampled_data_scenario1["centroid"] = sampled_data_scenario1["geometryn"].centroid

In [ ]:
#same centroid for each polygon
#sampled_data_scenario1.head(4)

In [113]:
## ALTERNATIVE: Group by NAME(district)
sampled_data_scenario_grouped = sampled_data_scenario2[['otherNAME','geometry','NAME','centroid']].groupby(['NAME'], as_index=False).agg({'otherNAME':'first','NAME': 'count', 'centroid':'first', 'geometry': 'first'})


In [114]:
# 3857 is the projected CRS of Shenzhen china, while 4326 is the geographic CRS. Here we need the projected CRS for centroid

sampled_data_scenario_grouped  = gpd.GeoDataFrame(sampled_data_scenario_grouped, crs="EPSG:3857", geometry=sampled_data_scenario_grouped.geometry)

In [115]:
sampled_data_scenario_grouped.shape

(6, 4)

In [116]:
sampled_data_scenario_grouped.head(2)

,otherNAME,NAME,centroid,geometry
0,南山区,68281,POINT (12683172.678 2577481.685),POINT (114.009 22.588)
1,宝安区,113747,POINT (12680354.419 2595697.733),POINT (114.032 22.633)


In [117]:
sampled_data_scenario_grouped.head(2)

,otherNAME,NAME,centroid,geometry
0,南山区,68281,POINT (12683172.678 2577481.685),POINT (114.009 22.588)
1,宝安区,113747,POINT (12680354.419 2595697.733),POINT (114.032 22.633)


In [118]:
sampled_data_scenario_grouped.sort_values('NAME',ascending=False)

,otherNAME,NAME,centroid,geometry
3,福田区,474657,POINT (12695031.079 2577223.135),POINT (114.064 22.543)
4,罗湖区,192505,POINT (12706355.523 2581018.763),POINT (114.103 22.582)
1,宝安区,113747,POINT (12680354.419 2595697.733),POINT (114.032 22.633)
0,南山区,68281,POINT (12683172.678 2577481.685),POINT (114.009 22.588)
5,龙岗区,47214,POINT (12729987.892 2588957.163),POINT (114.148 22.615)
2,盐田区,6403,POINT (12720500.796 2583286.511),POINT (114.273 22.591)


In [ ]:
# to numpy array
#count_scenario2 = sampled_data_scenario_grouped['geohash'].to_numpy()
#count_scenario2

In [119]:
# to numpy array
count_scenario2 = sampled_data_scenario_grouped['NAME'].to_numpy()
count_scenario2


array([ 68281, 113747,   6403, 474657, 192505,  47214])

In [120]:
count_scenario2 = count_scenario2/count_scenario2.sum()
count_scenario2

array([0.07563189, 0.1259926 , 0.00709232, 0.52575689, 0.21322941,
       0.05229689])

#MAPE

(by how many percent does each original tile diverge from its scenario equivalent

In [121]:
#Defining MAPE function
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [122]:
type(count_original)

numpy.ndarray

# END MAPE

## RMSE

In [123]:
#Defining RMSE function
def RMSE(Y_actual,Y_Predicted):
    rmse = np.sqrt(1/np.count_nonzero(Y_actual) * np.sum(np.power(Y_actual - Y_Predicted,2)))
    return rmse

## End RMSE

In [124]:
RMSE(count_original,count_scenario2)

0.05863680106908676

#point biserial correlation coefficient

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pointbiserialr.html
Like other correlation coefficients, this one varies between -1 and +1 with 0 implying no correlation. Correlations of -1 or +1 imply a determinative relationship.

In [125]:
from scipy import stats
stats.pointbiserialr(count_original, count_scenario2)


SignificanceResult(statistic=0.9459105786504935, pvalue=0.004309374475856013)

#end point biserial correlation coefficient

# Spearman correlation coefficient

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html

shows no difference between the two distributions - highly correlated 1

In [126]:
res = stats.spearmanr(count_original, count_scenario2)
res.statistic

0.8285714285714287

#end Spearman correlation coefficient

## Minkowski distance

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.minkowski.html

In [127]:
from scipy.spatial import distance
distance.minkowski(count_original, count_scenario2, 1)


0.24290518430413713

##end Minkowski distance

## Kullback–Leibler divergence


In [128]:
from scipy.special import rel_entr

#calculate (P || Q)
sum(rel_entr(count_original, count_scenario2))

0.07565915569652996

## end Kullback–Leibler divergence


## jensenshannon

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.jensenshannon.html

In [129]:
from scipy.spatial import distance
distance.jensenshannon(count_original, count_scenario2)

0.1270649759887051

also this one fail to capture despite being better than KL

= 0.0000011612128751813373
(real number)

https://www.calculatorsoup.com/calculators/math/scientific-notation-converter.php

## end jensenshannon

In [130]:
def signature_opt_NAME(gdf, crs):

    #centroids = gdf.geometry.to_crs(crs)#.centroid
    centroids = gdf.centroid#.to_crs(crs)
    sig = np.empty((len(gdf), 3), dtype=np.float32)
    # float32 needed as input for cv2.emd!

    # we need to normalize the data in case the total
    # n of the two compared distributions are not equal
    #sig[:,0] = gdf.geohash /gdf.geohash.sum()
    #mean instead of count
    sig[:,0] = gdf.NAME /gdf.NAME.sum()
    #    sig[:,0] = gdf.speed # /gdf.speed.mean()
    print(np.sum(sig[:,0]))
    sig[:,1] = centroids.x
    print(centroids.x)
    sig[:,2] = centroids.y
    print(centroids.y)
    return sig

In [131]:
sig_original = signature_opt_NAME(sampled_data_original_grouped, 3857 )
sig_original

0.99999994
0    113.979698
1    114.039001
2    114.216103
3    114.031799
4    114.099602
5    114.111099
dtype: float64
0    22.522900
1    22.607700
2    22.557199
3    22.524799
4    22.561501
5    22.598499
dtype: float64


array([[1.97084486e-01, 1.13979698e+02, 2.25228996e+01],
       [1.09438844e-01, 1.14039001e+02, 2.26077003e+01],
       [6.16008090e-03, 1.14216103e+02, 2.25571995e+01],
       [4.56678122e-01, 1.14031799e+02, 2.25247993e+01],
       [1.85212865e-01, 1.14099602e+02, 2.25615005e+01],
       [4.54256162e-02, 1.14111099e+02, 2.25984993e+01]], dtype=float32)

In [ ]:
#sig_original = signature_opt1(sampled_data_original_grouped, 3857 )
#sig_original

In [ ]:
## manually
'''sig_original = np.array([[1.97084486e-01, 1.13979698e+02, 2.25228996e+01],
       [1.09438844e-01, 1.14039001e+02, 2.26077003e+01],
       [6.16008090e-03, 1.14216103e+02, 2.25571995e+01],
       [4.56678122e-01, 1.14031799e+02, 2.25247993e+01],
       [1.85212865e-01, 1.14099602e+02, 2.25615005e+01],
       [4.54256162e-02, 1.14111099e+02, 2.25984993e+01]],np.float32)'''

'sig_original = np.array([[1.97084486e-01, 1.13979698e+02, 2.25228996e+01],\n       [1.09438844e-01, 1.14039001e+02, 2.26077003e+01],\n       [6.16008090e-03, 1.14216103e+02, 2.25571995e+01],\n       [4.56678122e-01, 1.14031799e+02, 2.25247993e+01],\n       [1.85212865e-01, 1.14099602e+02, 2.25615005e+01],\n       [4.54256162e-02, 1.14111099e+02, 2.25984993e+01]],np.float32)'

In [132]:
np.sum(sig_original[:,0])

0.99999994

In [ ]:
#sig_scen1 = signature_opt1(sampled_data_scenario_grouped, 3857 )
#sig_scen1

In [133]:
sig_scen1 = signature_opt_NAME(sampled_data_scenario_grouped, 3857 )
sig_scen1

1.0
0    114.009399
1    114.032204
2    114.272903
3    114.064400
4    114.102501
5    114.148499
dtype: float64
0    22.588200
1    22.633499
2    22.591499
3    22.542601
4    22.582500
5    22.615499
dtype: float64


array([[7.563189e-02, 1.140094e+02, 2.258820e+01],
       [1.259926e-01, 1.140322e+02, 2.263350e+01],
       [7.092324e-03, 1.142729e+02, 2.259150e+01],
       [5.257569e-01, 1.140644e+02, 2.254260e+01],
       [2.132294e-01, 1.141025e+02, 2.258250e+01],
       [5.229689e-02, 1.141485e+02, 2.261550e+01]], dtype=float32)

In [ ]:
##manually
'''sig_scen1 = np.array([[1.7187029e-01, 1.13979698e+02, 2.25228996e+01],
       [1.1810414e-01, 1.14039001e+02, 2.26077003e+01],
       [6.6479710e-03, 1.14216103e+02, 2.25571995e+01],
       [4.9283805e-01, 1.14031799e+02, 2.25247993e+01],
       [1.6151747e-01, 1.14099602e+02, 2.25615005e+01],
       [4.9022060e-02, 1.14111099e+02, 2.25984993e+01]],np.float32)'''

'sig_scen1 = np.array([[1.7187029e-01, 1.13979698e+02, 2.25228996e+01],\n       [1.1810414e-01, 1.14039001e+02, 2.26077003e+01],\n       [6.6479710e-03, 1.14216103e+02, 2.25571995e+01],\n       [4.9283805e-01, 1.14031799e+02, 2.25247993e+01],\n       [1.6151747e-01, 1.14099602e+02, 2.25615005e+01],\n       [4.9022060e-02, 1.14111099e+02, 2.25984993e+01]],np.float32)'

In [134]:
# cv2.DIST_L2 : the simple euclidean distance
# https://docs.opencv.org/3.4/d6/dc7/group__imgproc__hist.html#ga902b8e60cc7075c8947345489221e0e0
# https://docs.opencv.org/3.4/d7/d1b/group__imgproc__misc.html#gaa2bfbebbc5c320526897996aafa1d8eb
emd_scen1, _ , flow = cv2.EMD( sig_scen1,sig_original, distType = cv2.DIST_L2)

In [135]:
flow

array([[0.07563189, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.01655375, 0.10943884, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00616008, 0.        , 0.00093224,
        0.        ],
       [0.06907877, 0.        , 0.        , 0.45667812, 0.        ,
        0.        ],
       [0.03582007, 0.        , 0.        , 0.        , 0.17740934,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.00687128,
        0.04542562]], dtype=float32)

In [136]:
print("Geo movers distance scenario 1 (CRS: EPSG:3857): " + str(emd_scen1) + " meters") #try round(emd_scen1)


Earth movers distance scenario 1 (CRS: EPSG:3857): 0.04493200406432152 meters
